### Simple benchmark using wifi features and lightgbm 

Shows the use of wifi features I made to predict phone position. There is a lot of room for improvement, and for people interested in hyperparameter etc these features are an easy way to get started on this competition. Wifi features are available in [this dataset](https://www.kaggle.com/ammarali32/indoor-wifi-features). See this [forum post](https://www.kaggle.com/c/indoor-location-navigation/discussion/215445) for information on the approach. The code to generate the features is available in [this notebook](https://www.kaggle.com/devinanzelmo/wifi-features)

The Main Difference Between the datasets that mine includes the features using only wifi's that had 1000 or more bssid in the training dataset.
If you found it helpful upvote the original notebook.

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import glob
import os

In [ ]:
feature_dir = "../input/indoor-wifi-features/"

In [ ]:
# the metric used in this competition
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

# get our train and test files
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
ssubm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv', index_col=0)

In [ ]:
predictions = list()

for e, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[e], index_col=0)

    # simple grid search for tuning using path and groupkfold cv
    #for i in [127]:
    #    for n in [75]:
    #        modely = lgb.LGBMRegressor(
    #        n_estimators=n, num_leaves=i, n_jobs=1)
    #        gkf = model_selection.GroupKFold()
    #        scores = model_selection.cross_val_score(
    #            modely, x_train, y_trainy, scoring='neg_mean_squared_error',
    #            cv=gkf, groups=data.iloc[:,-1], n_jobs=5)
    #        print(i, n, l, scores, scores.mean())
    

    x_train = data.iloc[:,:-3]
    y_trainy = data.iloc[:,-2]
    y_trainx = data.iloc[:,-3]
    y_trainf = data.iloc[:,-1]

    modely = lgb.LGBMRegressor(
        n_estimators=125, num_leaves=90)
    modely.fit(x_train, y_trainy)

    modelx = lgb.LGBMRegressor(
        n_estimators=125, num_leaves=90)
    modelx.fit(x_train, y_trainx)

    modelf = lgb.LGBMClassifier(
        n_estimators=125, num_leaves=90)
    modelf.fit(x_train, y_trainf)
    
    test_predsx = modelx.predict(test_data.iloc[:,:-1])
    test_predsy = modely.predict(test_data.iloc[:,:-1])
    test_predsf = modelf.predict(test_data.iloc[:,:-1])
    
    test_preds = pd.DataFrame(np.stack((test_predsf, test_predsx, test_predsy))).T
    test_preds.columns = ssubm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)

In [ ]:
# generate prediction file 
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(ssubm.index)
all_preds.to_csv('submission.csv')